In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Input
import numpy as np
from matplotlib import pyplot as plt
import helper #courtesy of CS109B

In [3]:
# Ensure reproducible results
np.random.seed(109)
tf.random.set_seed(109)

### Image Generator

In [4]:
img_generator = helper.get_generator()

Found 4006 images belonging to 2 classes.


In [5]:
TARGET_SIZE = img_generator.target_size
print(f'Generator produces images of size {TARGET_SIZE} (with 3 color channels)')

Generator produces images of size (150, 150) (with 3 color channels)


In [6]:
BATCH_SIZE = img_generator.batch_size
print(f'Images are generated in batches of size {BATCH_SIZE}')

Images are generated in batches of size 16


### Feed-Forward Neural Network

In [7]:
FFNN = Sequential()
FFNN.add(Flatten(input_shape=(150, 150, 3))) # compare input_shape to TARGET_SIZE above
# specify a list of n_nodes for each dense layer [x,y,z,..] where x,y,z are ints
for n_nodes in [400, 128]:
    FFNN.add(Dense(n_nodes, activation='relu'))
FFNN.add(Dense(1, activation='sigmoid'))

FFNN.compile(loss='binary_crossentropy', metrics=['accuracy'])
FFNN.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 67500)             0         
_________________________________________________________________
dense (Dense)                (None, 400)               27000400  
_________________________________________________________________
dense_1 (Dense)              (None, 128)               51328     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 129       
Total params: 27,051,857
Trainable params: 27,051,857
Non-trainable params: 0
_________________________________________________________________


In [8]:
FFNN_history = FFNN.fit(
        img_generator,
        steps_per_epoch=150// BATCH_SIZE,
        epochs=10,
        validation_data=img_generator,
        validation_steps=50// BATCH_SIZE)

Train for 9 steps, validate for 3 steps
Epoch 1/10
9/9 [==============================] - 5s 575ms/step - loss: 68.7832 - accuracy: 0.5417 - val_loss: 24.0071 - val_accuracy: 0.6250
Epoch 2/10
9/9 [==============================] - 4s 423ms/step - loss: 25.5466 - accuracy: 0.5347 - val_loss: 8.1561 - val_accuracy: 0.6250
Epoch 3/10
9/9 [==============================] - 4s 402ms/step - loss: 14.9018 - accuracy: 0.4236 - val_loss: 7.4195 - val_accuracy: 0.6250
Epoch 4/10
9/9 [==============================] - 4s 407ms/step - loss: 5.7613 - accuracy: 0.5694 - val_loss: 4.4644 - val_accuracy: 0.6250
Epoch 5/10
9/9 [==============================] - 4s 406ms/step - loss: 7.0789 - accuracy: 0.5486 - val_loss: 8.7840 - val_accuracy: 0.6250
Epoch 6/10
9/9 [==============================] - 5s 547ms/step - loss: 7.6460 - accuracy: 0.4861 - val_loss: 3.8246 - val_accuracy: 0.3750
Epoch 7/10
9/9 [==============================] - 6s 643ms/step - loss: 5.2406 - accuracy: 0.5972 - val_loss: 2.4520

In [9]:
_, FFNN_acc = FFNN.evaluate(img_generator, steps=20)
print(f'FFNN Test Accuracy: {FFNN_acc}')

20/20 [==============================] - 4s 201ms/step - loss: 2.5096 - accuracy: 0.4531
FFNN Test Accuracy: 0.453125


### CNN

In [10]:
CNN = Sequential()
CNN.add(Input(shape=(150, 150, 3)))
for n_filters in [4,8,16,32, 64]:
    CNN.add(Conv2D(n_filters, kernel_size=3, activation='relu'))
    CNN.add(MaxPool2D())
CNN.add(Flatten())
CNN.add(Dense(64, activation='relu'))
CNN.add(Dense(1, activation='sigmoid'))
    
CNN.compile(loss='binary_crossentropy', metrics=['accuracy'])
CNN.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 4)       112       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 4)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 8)         296       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 8)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 16)        1168      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 16)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 32)       

In [11]:
CNN_history = CNN.fit(
        img_generator,
        steps_per_epoch=150 // BATCH_SIZE,
        epochs=10,
        validation_data=img_generator,
        validation_steps=50// BATCH_SIZE)
CNN.save_weights('second_try.h5')  # always save your weights after training or during training

Train for 9 steps, validate for 3 steps
Epoch 1/10
9/9 [==============================] - 11s 1s/step - loss: 0.6929 - accuracy: 0.5417 - val_loss: 0.6704 - val_accuracy: 0.7708
Epoch 2/10
9/9 [==============================] - 18s 2s/step - loss: 0.6601 - accuracy: 0.6944 - val_loss: 0.5980 - val_accuracy: 0.7500
Epoch 3/10
9/9 [==============================] - 5s 562ms/step - loss: 0.4786 - accuracy: 0.8125 - val_loss: 0.7353 - val_accuracy: 0.5000
Epoch 4/10
9/9 [==============================] - 9s 1s/step - loss: 0.3897 - accuracy: 0.8264 - val_loss: 0.1859 - val_accuracy: 0.9375
Epoch 5/10
9/9 [==============================] - 8s 908ms/step - loss: 0.2325 - accuracy: 0.9236 - val_loss: 0.0572 - val_accuracy: 1.0000
Epoch 6/10
9/9 [==============================] - 4s 450ms/step - loss: 0.1227 - accuracy: 0.9583 - val_loss: 0.1126 - val_accuracy: 0.9583
Epoch 7/10
9/9 [==============================] - 4s 452ms/step - loss: 0.1199 - accuracy: 0.9583 - val_loss: 0.0667 - val_accu

In [12]:
_, CNN_acc = CNN.evaluate(img_generator, steps=20)
print(f'CNN Test Accuracy: {CNN_acc}')

20/20 [==============================] - 4s 194ms/step - loss: 0.0186 - accuracy: 1.0000
CNN Test Accuracy: 1.0
